In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations, chain

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1156,tt0299930,54000000,7266209,Gigli,Ben Affleck|Jennifer Lopez|Justin Bartha|Laini...,Martin Brest,Life doesn't play by the rules.,Gigli is ordered to kidnap the psychologically...,121,Drama,City Light Films|Revolution Studios|Casey Silv...,8/1/2003,3.8,2003
1234,tt2364841,30000000,62616646,Runner Runner,Ben Affleck|Gemma Arterton|Justin Timberlake|A...,Brad Furman,The house always wins.,When a poor college student who cracks an onli...,91,Crime|Thriller|Drama,20th Century Fox|Double Feature Films|Appian W...,9/24/2013,5.5,2013
604,tt1099212,37000000,392616625,Twilight,Kristen Stewart|Robert Pattinson|Billy Burke|T...,Catherine Hardwicke,"When you can live forever, what do you live for?",When Bella Swan moves to a small town in the P...,122,Adventure|Fantasy|Drama|Romance,Summit Entertainment|Maverick Films|Imprint En...,11/20/2008,5.8,2008
331,tt0913354,20000000,22942221,Armored,Columbus Short|Laurence Fishburne|Skeet Ulrich...,NimrÃ³d Antal,Who will be the last man standing?,A crew of officers at an armored transport sec...,88,Action|Crime|Drama|Thriller,Screen Gems|Stars Road Entertainment|Farah Fil...,12/4/2009,5.6,2009
273,tt1121931,20000000,34560577,Crank: High Voltage,Jason Statham|Amy Smart|Dwight Yoakam|Efren Ra...,Mark Neveldine|Brian Taylor,"Stay Charged, Stay Alive!",Chelios faces a Chinese mobster who has stolen...,96,Action|Thriller|Crime,Lakeshore Entertainment|Lionsgate|Radical Media,4/16/2009,5.9,2009


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [75]:
def splitting(col):
    ''' Создание копии датасета, разделение данных в колонке col по "|"
    с расширением датасета путём добавления разделённых данных в новые строки.

    '''
    data2 = data.copy()
    data2[col] = data2[col].str.split('|')
    data2 = data2.explode(col)
    return data2


answers = {} # Создадим словарь для ответов

# Преобразуем дату в формат datetime
data['release_date'] = pd.to_datetime(data['release_date'])

# Добавим месяц релиза в датасет
data['release_month'] = data['release_date'].dt.month

data['profit'] = data['revenue'] - data['budget'] # Добавим колонку profit

# Датасеты с разделёнными актёрами, жанрами, киностудиями и режиссёрами
casts = splitting('cast')
genres = splitting('genres')
companies = splitting('production_companies')
directors = splitting('director')


# 1. У какого фильма из списка самый большой бюджет?

In [40]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [ ]:
# Все 4 варианта подходят для решения аналогичных задач

In [5]:
data[data['budget'] == data['budget'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [6]:
(data.sort_values('budget', ascending=False).head(1)
 [['original_title', 'imdb_id']])

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 3

In [7]:
(data.groupby(['original_title', 'imdb_id'])
 ['budget'].max().sort_values(ascending=False).head(1))

original_title                               imdb_id  
Pirates of the Caribbean: On Stranger Tides  tt1298650    380000000
Name: budget, dtype: int64

ВАРИАНТ 4

In [8]:
data.nlargest(1, 'budget')[['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [41]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [9]:
data.nlargest(1, 'runtime')[['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [42]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [10]:
data.nsmallest(1, 'runtime')[['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [43]:
answers['4'] = '110'

In [11]:
round(data['runtime'].mean(), 0)

110.0

# 5. Каково медианное значение длительности фильмов? 

In [44]:
answers['5'] = '107'

In [12]:
round(data['runtime'].median(), 0)

107.0

# 6. Какой самый прибыльный фильм?

In [45]:
answers['6'] = 'Avatar (tt0499549)'

In [13]:
data.nlargest(1, 'profit')[['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [46]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [14]:
data.nsmallest(1, 'profit')[['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [47]:
answers['8'] = '1478'

In [15]:
data[data['profit'] > 0]['original_title'].count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [48]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [16]:
data_2008 = data.query('release_year == 2008')
data_2008.nlargest(1, 'revenue')[['original_title', 'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [49]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [18]:
data_2014 = data.query('(release_year>=2012) & (release_year<=2014)')
data_2014.nsmallest(1, 'profit')[['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819



# 11. Какого жанра фильмов больше всего?

In [50]:
answers['11'] = 'Drama'

In [19]:
Counter(data['genres'].str.split('|').sum()).most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [51]:
answers['12'] = 'Drama'

In [20]:
profit_movies = data.query('profit > 0')
Counter(profit_movies['genres'].str.split('|').sum()).most_common(1)

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [52]:
answers['13'] = 'Peter Jackson'

In [22]:
data.groupby('director')['revenue'].sum().nlargest(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [53]:
answers['14'] = 'Robert Rodriguez'

In [23]:
(directors[directors['genres'].str.contains('Action')]
 ['director'].value_counts())

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Des McAnuff           1
Betty Thomas          1
Albert Hughes         1
Ben Garant            1
David Leland          1
Name: director, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [54]:
answers['15'] = 'Chris Hemsworth'

In [24]:
data_2012 = casts.query('release_year == 2012')
data_2012.groupby('cast')['revenue'].sum().nlargest(1)

cast
Chris Hemsworth    2027450773
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [55]:
answers['16'] = 'Matt Damon'

In [25]:
hight_budget = casts[casts['budget'] > casts['budget'].mean()]
hight_budget.groupby('cast')['budget'].count().nlargest(1)

cast
Matt Damon    18
Name: budget, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [56]:
answers['17'] = 'Action'

In [26]:
genres[genres['cast'].str.contains('Nicolas Cage')]['genres'].value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Mystery             3
Animation           3
History             2
Romance             1
War                 1
Horror              1
Name: genres, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [57]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [27]:
Paramount = companies.query('production_companies == "Paramount Pictures"')
Paramount.nsmallest(1, 'profit')[['original_title', 'imdb_id']]

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [58]:
answers['19'] = '2015'

In [28]:
data.groupby('release_year')['revenue'].sum().nlargest(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [59]:
answers['20'] = '2014'

In [29]:
Warner = data[data['production_companies'].str.contains('Warner Bros')]
Warner.groupby('release_year')['revenue'].sum().nlargest(1)

release_year
2014    3243064519
Name: revenue, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [60]:
answers['21'] = 'Сентябрь'

In [74]:
pivot = data.pivot_table(
       values=['original_title'],
       index=['release_year'],
       columns=['release_month'],
       aggfunc='count',
       fill_value=0,
       margins=True
)
pivot.loc['All'].sort_values(ascending=False)

                release_month
original_title  All              1889
                9                 227
                12                190
                10                186
                8                 161
                3                 156
                4                 149
                6                 147
                11                146
                7                 142
                5                 140
                2                 135
                1                 110
Name: All, dtype: int64

ВАРИАНТ 2

In [31]:
data.groupby('release_month')['original_title'].count().nlargest(1)

release_month
9    227
Name: original_title, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [61]:
answers['22'] = '450'

In [32]:
(data.query('(release_month<9) & (release_month>5)')
 ['original_title'].value_counts().sum())

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [62]:
answers['23'] = 'Peter Jackson'

In [33]:
pivot = data.loc[data['release_month'].isin([1, 2, 12])].pivot_table(
       values=['original_title'],
       index=['release_month'],
       columns=['director'],
       aggfunc='count',
       fill_value=0,
       margins=True
)
pivot.loc['All'].sort_values(ascending=False)

                director                           
original_title  All                                    435
                Peter Jackson                            7
                Clint Eastwood                           6
                Steven Soderbergh                        6
                Adam Shankman                            4
                                                      ... 
                Matt Bettinelli-Olpin|Tyler Gillett      1
                Matt Reeves                              1
                Matthew O'Callaghan                      1
                Matthew Vaughn                           1
                Aaron Seltzer|Jason Friedberg            1
Name: All, Length: 333, dtype: int64

ВАРИАНТ 2

In [35]:
winter = data.query('(release_month<3) | (release_month==12)')
winter.groupby('director')['original_title'].count().nlargest(1)

director
Peter Jackson    7
Name: original_title, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [63]:
answers['24'] = 'Four By Two Productions'

In [36]:
companies['len_title'] = companies['original_title'].apply(lambda x: len(x))
companies.groupby('production_companies')['len_title'].mean().nlargest(1)

production_companies
Four By Two Productions    83.0
Name: len_title, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [64]:
answers['25'] = 'Midnight Picture Show'

In [73]:
# Разделил строку с помощью "\", потому что при заключении кода в собки,
# он отказывается работать: ошибка invalid syntax.
companies['len_overview'] = \
 companies['overview'].apply(lambda x: len(x.split(' ')))
companies.groupby('production_companies')['len_overview'].mean().nlargest(1)

production_companies
Midnight Picture Show    175.0
Name: len_overview, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [65]:
answers['26'] = 'The Dark Knight, Inside Out, 12 Years a Slave'

In [70]:
(data[data['vote_average'] >= data.groupby('original_title')['vote_average'].
 sum().quantile(0.99)][['original_title', 'vote_average']].
 sort_values('vote_average', ascending=False))

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
370,Inception,7.9
1191,12 Years a Slave,7.9
1183,The Wolf of Wall Street,7.9
1081,The Lord of the Rings: The Return of the King,7.9
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [66]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

In [39]:
data['cast_split'] = data['cast'].str.split('|')
data['comb'] = data['cast_split'].apply(lambda x: list(combinations(x, 2)))
together = list(chain(*data['comb'].tolist()))
Counter(together).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

# Submission

In [67]:
# Ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'The Dark Knight, Inside Out, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [68]:
# Общее количество вопросов
len(answers)

27